# UNET Model for Segmentation


In [ ]:
from importsfunctions import *

## Parameters

In [ ]:
complexity1=32
act1='relu'
batch=2
loss1='binary_crossentropy'
dropout_c1=True
optimizer1=Adam(learning_rate=1e-4)#,amsgrad=True )
DataAug=True

patience1=5 ## Sem Adaptive LR
log_dire = os.path.join(
    "logs",
    "Segm","testa"
)
filepath="Segmentation_teste.h5"

### Callbacks

In [ ]:
checkpointer = ModelCheckpoint(filepath, verbose=1,monitor='val_jaccard', save_weights_only=True,save_best_only=True)
Reduce=ReduceLROnPlateau(factor=0.7, patience=patience1, min_lr=1e-9, verbose=1)
earlystopper = EarlyStopping(patience=50,monitor='val_jaccard', verbose=1)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dire, histogram_freq=1)

## Data import

In [ ]:
#Validation=True significa que não há data Augmentation
# Parameters
paramsT = {'batch_size': batch,
          'n_classes': 3,
          'shuffle': True,
         'validation': not(DataAug)}
paramsV = {'batch_size': batch,
          'n_classes': 3,
          'shuffle': False,
         'validation': True}

# Datasets
DATAPATH_csv='ISIC-2017_Training_Part3_GroundTruth_r.csv'
#DATAPATH_csv='ISIC-2017_Training_Part3_GroundTruth.csv'#sem data augmentation
DATAPATH_segm_csv='ISIC-2017_Validation_Part3_GroundTruth.csv'
partition,labels = create_dictionary(DATAPATH_csv,DATAPATH_segm_csv)

# Generators
training_generator = DataGenerator_segmentation(partition['train'], labels, **paramsT)
validation_generator = DataGenerator_segmentation(partition['validation'], labels, **paramsV)

## Build model

In [ ]:
model=redeUNET(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS,act=act1,complexity=complexity1,dropout_c=dropout_c1)
model.compile(optimizer=optimizer1, loss=loss1 ,metrics=[jaccard])
model.summary()

## Actual training

In [ ]:
results=model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    steps_per_epoch=2000/batch,        
                    workers=6,
                    epochs=10,
                    callbacks=[checkpointer,Reduce,earlystopper,tensorboard_callback],
                    verbose=1)

## Load Weights

In [ ]:
model=redeUNET(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS,act=act1,complexity=complexity1,dropout_c=dropout_c1)
#model.compile(optimizer=optimizer1, loss=loss1 ,metrics=[jaccard])
#model.load_weights(filepath)
model.load_weights("Weights\Segmentation\Segmentation7_32_2.h5", by_name=True)

## Generate Predictions and evaluate

In [ ]:
DATAPATH = 'ISIC2017/ISIC-2017_Validation_Data/ISIC-2017_Validation_Data_Resize_5721/'
DATAPATH_segm = 'ISIC2017/ISIC-2017_Validation_Part1_GroundTruth/ISIC-2017_Validation_Part1_GroundTruth_Resize_5721/'
DATAPATH_csv='ISIC2017/ISIC-2017_Validation_Part3_GroundTruth.csv'
x_validation,y_validation,y_validationclass,nomefichv=import_datasetBSrot(DATAPATH,DATAPATH_segm, DATAPATH_csv,150)
x_validation=x_validation/255

In [ ]:
predictions_val = model.predict(x_validation,batch_size=4)
preds_val_t = (predictions_val > 0.5).astype(np.uint8)
DATAPATH_OUT = 'ISIC2017/ISIC-2017_Validation_Part1_GroundTruth/Result/teste/1/'
automatePostProcess(preds_val_t,DATAPATH_OUT,nomefichv)

## Prediction example

In [ ]:
ix = random.randint(0, 150)
imshow(x_validation[ix])
print(ix)
plt.show()
imshow(np.squeeze(y_validation[ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()
print('IOU=', getIOU(preds_val_t[ix], y_validation[ix]))

# Test Dataset

In [ ]:
model=redeUNET(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS,act=act1,complexity=complexity1,dropout_c=dropout_c1)
model.compile(optimizer=optimizer1, loss=loss1 ,metrics=[jaccard])
#model.load_weights(filepath)
model.load_weights("Weights\Segmentation\Segmentation7_32_2.h5", by_name=True)

In [ ]:
DATAPATH = 'ISIC2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data_Resize_5721/'
DATAPATH_segm = 'ISIC2017/ISIC-2017_Test_v2_Part1_GroundTruth/ISIC-2017_Test_v2_Part1_GroundTruth_Resize_5721/'
DATAPATH_csv='ISIC2017/ISIC-2017_Test_v2_Part3_GroundTruth.csv'
x_validation,y_validation,y_validationclass,nomefichv=import_datasetBSrot(DATAPATH,DATAPATH_segm, DATAPATH_csv,600)
x_validation=x_validation/255

In [ ]:
predictions_val = model.predict(x_validation,batch_size=4)
preds_val_t = (predictions_val > 0.5).astype(np.uint8)
DATAPATH_OUT = 'ISIC2017/ISIC-2017_Test_v2_Part1_GroundTruth/Result/teste/1/'
automatePostProcessTest(preds_val_t,DATAPATH_OUT,nomefichv)

In [ ]:
ix = random.randint(0, 600)
imshow(x_validation[ix])
print(ix)
plt.show()
imshow(np.squeeze(y_validation[ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()
print('IOU=', getIOU(preds_val_t[ix], y_validation[ix]))